# LENDING CLUB CASE STUDY

In [475]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## 1. Data Sourcing

#### Load `Data_Dictionary.xlsx`

In [476]:
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

pd.read_excel("Data_Dictionary.xlsx", header=0)

,LoanStatNew,Description
0,acc_now_delinq,The number of accounts on which the borrower is now delinquent.
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan application
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by the borrower during registration.
...,...,...
112,verification_status,"Indicates if income was verified by LC, not verified, or if the income source was verified"
113,verified_status_joint,"Indicates if the co-borrowers' joint income was verified by LC, not verified, or if the income source was verified"
114,zip_code,The first 3 numbers of the zip code provided by the borrower in the loan application.
115,NaN,NaN


#### Load -  `loan.csv`

In [477]:
loan = pd.read_csv('loan.csv', dtype={'next_pymnt_d': object})

## 2. Data Understanding

In [478]:
print(loan.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB
None


In [479]:
print(loan.shape)

(39717, 111)


In [480]:
print(loan.columns)

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit'],
      dtype='object', length=111)


## 3. Data Cleaning

#### Step 3.1 - Remove Nulls columns over 90%

In [481]:
#Checking Null Values in percentage across the columns
loan.isna().sum()*100/len(loan)

id                              0.000000
member_id                       0.000000
loan_amnt                       0.000000
funded_amnt                     0.000000
funded_amnt_inv                 0.000000
                                 ...    
tax_liens                       0.098195
tot_hi_cred_lim               100.000000
total_bal_ex_mort             100.000000
total_bc_limit                100.000000
total_il_high_credit_limit    100.000000
Length: 111, dtype: float64

In [482]:
loan = loan.loc[:, loan.isna().sum() < len(loan)*.9]

In [483]:
loan.shape

(39717, 55)

In [484]:
loan.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt',
       'pub_rec_bankruptcies', 'tax_liens'],
      dtype='object')

#### Step 3.2: Remove duplicates based on ID column

In [485]:
loan['id'].duplicated().any() # No Duplicates

False

#### Step 3.3 Removing Columns

In [486]:
#Customer Behaviour Variables
loan.drop(["delinq_2yrs",
           "earliest_cr_line",
           "inq_last_6mths",
           "open_acc",
           "pub_rec",
           "revol_bal",
           "revol_util",
           "total_acc",
           "out_prncp",
           "out_prncp_inv",
           "total_pymnt",
           "total_pymnt_inv",
           "total_rec_prncp",
           "total_rec_int",
           "total_rec_late_fee", 
           "recoveries",
           "collection_recovery_fee",
           "last_pymnt_d",
           "last_pymnt_amnt",
           "last_credit_pull_d",
           "application_type"
           ],axis = 1, inplace=True)

In [487]:
loan.shape

(39717, 34)

In [488]:
loan.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'mths_since_last_delinq',
       'initial_list_status', 'collections_12_mths_ex_med', 'policy_code',
       'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt',
       'pub_rec_bankruptcies', 'tax_liens'],
      dtype='object')

In [489]:
# Removing Columns - Single Value in each column
loan.drop(["pymnt_plan",
           'collections_12_mths_ex_med',
           'policy_code',
           'acc_now_delinq',
           'chargeoff_within_12_mths',
           'delinq_amnt',
           'pub_rec_bankruptcies',
           'tax_liens'
          ],axis = 1, inplace=True)

In [490]:
# Removing Columns - Columns assinged when loan is approved
loan.drop(['id',
           'member_id',
           "emp_title",
           "url",
           'desc',
           'purpose',
           'title',
           "zip_code", # Incomplete zip code
           "mths_since_last_delinq",
           "initial_list_status"
          ],axis = 1, inplace=True)

In [491]:
# Removing Columns - Similar Columns
loan.drop(["funded_amnt", 
           "funded_amnt_inv"] ,axis = 1, inplace=True)

In [492]:
loan.shape

(39717, 14)

In [493]:
# Final List of columns to work on.
loan.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'addr_state', 'dti'],
      dtype='object')

#### Step 3.4 - Removing rows `loan_status` == `Current`

In [494]:
print(loan["loan_status"].value_counts())

Fully Paid     32950
Charged Off     5627
Current         1140
Name: loan_status, dtype: int64


In [495]:
loan = loan[loan["loan_status"] != 'Current']

In [496]:
loan.shape

(38577, 14)

#### Step 3.5 - Fixing `term` column (Removing `month`)

In [497]:
loan.term.unique()

array([' 36 months', ' 60 months'], dtype=object)

In [498]:
loan.term = loan.term.apply(lambda x: int(x.replace(" months", "")))

In [499]:
loan.term.value_counts()

36    29096
60     9481
Name: term, dtype: int64

#### Step 3.6 - Fixing `int_rate` column (Removing `%`)

In [500]:
loan.int_rate = loan.int_rate.apply(lambda x: float(x[:-1]))

In [501]:
loan.int_rate.head()

0    10.65
1    15.27
2    15.96
3    13.49
5     7.90
Name: int_rate, dtype: float64

#### Step 3.7 - Fixing column `emp_length` (removing `years` and `+`)  

In [502]:
loan.emp_length.unique()

array(['10+ years', '< 1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '1 year', '6 years', '2 years', '7 years',
       nan], dtype=object)

In [503]:
loan.emp_length = loan.emp_length.astype('str')

In [504]:
loan.emp_length = loan.emp_length.apply(lambda x: x.strip(" years"))

In [505]:
loan.emp_length.unique()

array(['10+', '< 1', '3', '8', '9', '4', '5', '1', '6', '2', '7', 'nan'],
      dtype=object)

In [506]:
def clean(input):
    if "nan" in input:
        return int(-1)
    elif "+" in input:
        return int(input.replace("+", ""))
    elif "<" in input:
        return int(input.replace("< ", ""))
    else:
        return int(input)

loan.emp_length = loan.emp_length.apply(clean)

In [507]:
loan.emp_length.unique()

array([10,  1,  3,  8,  9,  4,  5,  6,  2,  7, -1])

In [508]:
loan.verification_status.unique()

array(['Verified', 'Source Verified', 'Not Verified'], dtype=object)

In [509]:
loan.home_ownership.unique()

array(['RENT', 'OWN', 'MORTGAGE', 'OTHER', 'NONE'], dtype=object)

In [510]:
loan.loan_status.unique()

array(['Fully Paid', 'Charged Off'], dtype=object)

#### Step 3.8 - Deriving `issue_month` and `issue_year` from `issue_d`

In [511]:
from datetime import datetime
loan["issued_month"] = pd.DatetimeIndex(loan.issue_d.apply(lambda x : datetime.strptime(x, '%b-%y'))).month

In [512]:
loan.issued_month.unique()

array([12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1])

In [ ]:
loan["issued_year"] = pd.DatetimeIndex(loan.issue_d.apply(lambda x : datetime.strptime(x, '%b-%y'))).year

In [ ]:
loan.issued_year.unique()

In [ ]:
loan.drop(["issue_d"] ,axis = 1, inplace=True)

#### Step 3.9 - Driving numeric value from `sub_grade`

In [ ]:
loan.sub_grade = loan.sub_grade.astype("string")

In [ ]:
loan.sub_grade = loan.sub_grade.apply(lambda x: int(x[-1]))

In [ ]:
loan.columns

In [ ]:
loan.head()

## 4. Outlier Treatment

#### Step 4.1 - Treating `annual_inc`

In [ ]:
sns.boxplot(loan['annual_inc'])

In [ ]:
quantile_info = loan.annual_inc.quantile([0.5, 0.75,0.90, 0.95, 0.97,0.98, 0.99])
quantile_info

In [ ]:
annual_inc_99_per = loan['annual_inc'].quantile(0.99)
loan = loan[loan.annual_inc <= annual_inc_99_per]

In [ ]:
sns.boxplot(loan.annual_inc)

## 5. Univariant Analysis

#### Step 5.1 -  `loan_status` 

In [ ]:
sns.countplot(x = 'loan_status', data = loan)

#### Step 5.2 - `term` 

In [ ]:
sns.countplot(x = 'term', data = loan)

#### Step 5.3 - `home_ownership` 

In [ ]:
loan['home_ownership'].unique()

In [ ]:
#replacing 'NONE' with 'OTHERS'
loan['home_ownership'].replace(to_replace = ['NONE'],value='OTHER',inplace = True)

In [ ]:
loan['home_ownership'].unique()

In [ ]:
sns.countplot(x = 'home_ownership', data = loan)

#### Step 5.4 - `addr_state` 

In [ ]:
g = sns.countplot(x = 'addr_state', data = loan)
g.set_xticklabels(g.get_xticklabels(), rotation=90)

#### Step 5.5 -  `int_rate` 

In [ ]:
x = [7.5, 12.5, 17.5, 22.5]
labels = ['5 - 10', '11 - 15','16 - 20','21 - 25']
plt.hist(loan['int_rate'], bins = [5,10,15,20,25],rwidth = 0.7, color = "orange", edgecolor = 'k')
plt.xticks(x, labels, rotation ='vertical')
plt.show()

In [ ]:
plt.figure(figsize=(12,6), facecolor='Cyan')
plt.title('Univariate Analysis: Loan amount', fontdict={'fontsize': 15, 'fontweight' : 5, 'color' : 'black'})
sns.histplot(x = loan.loan_amnt)
xy_fontdict = {'fontsize':15, 'fontweight':5, 'color':'black'}
plt.xlabel('Loan Amount', fontdict=xy_fontdict)
plt.ylabel('Loam Amount Frequency', fontdict=xy_fontdict)
plt.show()

In [ ]:
loan['dti'].plot(kind='kde')
#Most loans are lent to borrowers with DTI ratio in between 10 and 20.

In [ ]:
sns.displot(loan.emp_length)
plt.xlabel('Employment length (in years)')

#### Step 5.6 Observation from Univariant Analysis
- Majority of the loans are fully paid, only about ~5500(15%) loans are charged-off
- Majority of the loan applications opts for 36 months loan term. About ~30,000 (75%)
- Minority of the folks who has loan are actually home owners (about ~2500 home owners only)
- Majority of the loan applications are from California State
- Majority of the loan amount are from ~5K to ~15K
- Most loans are lent to borrowers with DTI ratio in between 10 and 20.
- Majority of the home owners have either high work experience of 10 + years or 1 year. 

## 6. Segmented Univariate Analysis

In [ ]:
loan.groupby(["loan_status", 'home_ownership'])["loan_amnt"].agg(np.mean)

In [ ]:
loan.groupby(["loan_status", 'emp_length'])["loan_amnt"].agg(np.mean)

In [ ]:
loan.groupby(["loan_status"])["int_rate"].agg(np.mean)

In [ ]:
loan.groupby(["loan_status"])["dti"].agg(np.mean)

In [ ]:
loan.groupby(["loan_status"])["annual_inc"].agg(np.mean)

In [ ]:
loan.groupby(["loan_status"])["installment"].agg(np.mean)

In [ ]:
loan.groupby(["loan_status", "grade"])["loan_amnt"].agg(np.mean)

In [ ]:
loan.groupby(["loan_status", "home_ownership"])["loan_amnt"].agg(np.mean)

In [ ]:
loan.groupby(["loan_status", "verification_status"])["loan_amnt"].agg(np.mean)

#### Step 6.1 Observation from Segmented Univariate Analysis
- Loan amount for charged-off loans are higher than fully paid loans
- Interests rate are higher for charged off loans.
- DTI ration for charged-off loans are higher.
- Annual income for fully paid loan are higher.
- Interestingly, loan amount goes up with pooper grade.
- The loan amounts are higher for people with Mortage then people with own houses.
- The loan amounts are higer for verified individuals

## 7. Bivariate Analysis

#### Step 7.1 - Continous Variable - Bivariant Analysis

In [ ]:
plt.figure(figsize=(15,15))
cont_var= ['loan_amnt', 'int_rate', 'installment','annual_inc','dti', 'issued_year', 'issued_month', 'term', 'emp_length']
corr = loan[cont_var].corr()
g=sns.heatmap(corr, annot=True, center=0.5)
bottom, top = g.get_ylim()
g.set_ylim(bottom + 0.5,top - 0.5)

#### Step 7.2 - Categorical Variable - Bivariant Analysis

In [ ]:
pd.pivot_table(loan, index=["grade"],columns=['loan_status'], aggfunc=np.mean)

In [ ]:
pd.pivot_table(loan, index=["home_ownership"],columns=['loan_status'], aggfunc=np.mean)

In [ ]:
pd.pivot_table(loan, index=["term"],columns=['loan_status'], aggfunc=np.mean)

In [ ]:
pd.pivot_table(loan, index=["verification_status"],columns=['loan_status'], aggfunc=np.mean)

## 8. Recommendations from Case Study

The below analysis is for the chanrged-off loans. The probability for defaulting increases with following:
- If the applicant home ownership is mortgage and loan amount is greater than 14K.
- if the applicant employment duration is 10+ years and loan amount is greater than 14k.
- Higher interest rate with around 14% rate, increasses the default posibility.
- DTI ration of over 14, increasses the default posibility.
- Lower annual income of about 60K, increasses the default posibility.
- Grade G loan, with annual income of about 74K are more likely to default.
- Owners on Rent with about 50K annual income, and 14% interest are more likely to default.
- When the loan is verified and loan amount is over 16K and interest rate is around 15%.